In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch
from utils import dataset_gen, minmaxscaler
from model import GNN
import pandas as pd
import os
import seaborn as sns
from torch_geometric.loader import DataLoader

device = torch.device('cpu')

In [10]:
indicators = ['WSS','OSI']#

sub_ind = [indicators[0]]

task_type = 'extrap'
param_type = 'layers'
conv_type = 'gtr'
input_size = 7 # dataset features: Time, Press_SA, Press_abd, FlowRate, coord(x), coord(y), coord(z)
hidden_size = 32
layers_list = [2,3,4,5]
perc = 125
output_size = len(sub_ind)
dataset = dataset_gen(perc,sub_ind)
num_nodes = dataset[0].num_nodes
T = len(dataset)
loader = DataLoader(dataset, batch_size = 1, shuffle=False)

y = torch.zeros((T,num_nodes))
for i, data in enumerate(loader):
    y[i,:] = data.y.squeeze()
t = torch.linspace(0,1,T)

In [11]:
model_folder = '_'.join(['store', param_type, task_type, sub_ind[0]])+'/'
os.makedirs(model_folder, exist_ok=True)

In [5]:


nodes = [60, 735, 1167, 1083, 1415]
data = []
for i, num_layers in enumerate(layers_list):
    for it in range(5):       
        model_name ='_'.join(['pred',indicators[0],str(num_layers),str(it)+'.pt'])
        for node in nodes:
            for j in range(500):
                data.append({'num_layer': num_layers, 'it': it, 'node': node, 't':float(t[j]), 'pred': float(torch.load(model_folder+model_name).detach().numpy()[j,node])})
            #pred_plot[i,j,:,:] = torch.load('_'.join(['pred',indicators[0],str(num_layers),str(j)])).detach()
df = pd.DataFrame.from_records(data)

df.to_csv('layers_extrap_WSS.csv')

In [28]:
for node in nodes:
    fig, axes = plt.subplots(1,4, sharey=True, figsize=(16, 8) )
    axes[0].set_ylabel(sub_ind[0], fontsize = '16')
    for i,l in enumerate(layers_list):
        red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
        axes[i].set_xlabel('t', fontsize = '16')
        axes[i].tick_params(axis = 'both', which='minor', labelsize= 12)
        sns.lineplot(data = red, x="t", y="pred", ax=axes[i], label = 'Prediction')
        sns.lineplot(x = t, y = y[:,node].detach(),ax=axes[i], label = 'Ground Truth')
        axes[i].legend(fontsize = '14', loc = 'upper right')
        axes[i].set_title(str(l)+' layers')

    os.makedirs('seaborn/pred_layers_extrap/', exist_ok=True)
    plt.savefig('seaborn/pred_layers_extrap/pred'+str(node)+'.pdf')
    plt.close()

/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  red = df[df['num_layer']==l][df['node']==node][['t','pred','it']]
/var/folders/4g/4249y1xd42d75jmm55xxl99m0000gn/T/ipykernel_8613/747171996.py:6: UserWarning: Boolean Ser

TAWSS computation

In [12]:
errors = []
T = len(dataset)
dt = 1/T
data = minmaxscaler(np.load(str(perc)+'Percent.npz')['WSS'])
data[0,:] = data[0,:]/2
data[-1,:] = data[-1,:]/2
TAWSS = dt * np.sum(np.abs(data), axis = 0)
for it in range(5):
    for num_layers in layers_list:
        model_name ='_'.join(['pred',indicators[0],str(num_layers),str(it)+'.pt'])
        
        pred = torch.load(model_folder+model_name).detach().numpy()
        pred[0,:] /= 2
        pred[-1,:] /=2
        TAWSSp = dt * np.sum(np.abs(pred[:,:]), axis = 0)
        error = np.linalg.norm(TAWSS-TAWSSp,ord= np.inf)/np.linalg.norm(TAWSS, ord = np.inf)
        errors.append({'layer_num':num_layers, 'it':it, 'error': error})
db = pd.DataFrame.from_records(errors)

In [14]:
db.groupby('layer_num').describe()[['error']]

error                                                              \
          count      mean       std       min       25%       50%       75%   
layer_num                                                                     
2           5.0  0.559603  0.014433  0.541829  0.555461  0.555640  0.563897   
3           5.0  0.646569  0.113339  0.528463  0.576301  0.635530  0.667246   
4           5.0  0.653398  0.096201  0.557819  0.597313  0.607853  0.711891   
5           5.0  0.660333  0.055026  0.602390  0.621024  0.656140  0.679077   

                     
                max  
layer_num            
2          0.581192  
3          0.825306  
4          0.792113  
5          0.743036